In [ ]:
import os
import pandas as pd
import numpy as np
import pickle

In [ ]:
with open('dataframe_clean.pkl', 'rb') as f:
    df = pickle.load(f)

In [ ]:
df.columns

In [ ]:
from pgmpy.models.BayesianNetwork import BayesianNetwork
from pgmpy.estimators import StructureEstimator, BayesianEstimator, HillClimbSearch, PC
from sklearn.preprocessing import LabelEncoder

class BayesianModel(BayesianNetwork):

    def __init__(self, *, 
        ebunch=None, 
        graph_search_algo=HillClimbSearch, 
        scoring_method='k2score', 
        start_dag=None, 
        fixed_edges=set(), 
        tabu_length=100, 
        max_indegree=None, 
        black_list=None, 
        white_list=None, 
        epsilon=0.0001, 
        max_iter=1000000.0, 
        show_progress=True, 

        variant='stable',
        ci_test='chi_square',
        max_cond_vars=5,
        return_type='dag',
        significance_level=0.01,

        estimator=BayesianEstimator, 
        prior_type='BDeu', 
        pseudo_counts=[], 
        equivalent_sample_size=5
        ):
        
        if graph_search_algo == PC and return_type in ['pdag', 'cpdag']:
            raise ValueError('BayesianModel currently has no support for PDAG\'s')
        
        super().__init__(ebunch)
        
        self.ebunch = ebunch
        self.graph_search_algo = graph_search_algo
        self.scoring_method = scoring_method
        self.start_dag = start_dag
        self.fixed_edges = fixed_edges
        self.tabu_length = tabu_length
        self.max_indegree = max_indegree
        self.black_list = black_list
        self.white_list = white_list
        self.epsilon = epsilon
        self.max_iter = max_iter
        self.show_progress = show_progress

        self.variant = variant
        self.ci_test = ci_test
        self.max_cond_vars = max_cond_vars
        self.return_type = return_type
        self.significance_level = significance_level

        self.estimator = estimator
        self.prior_type = prior_type
        self.pseudo_counts = pseudo_counts
        self.equivalent_sample_size = equivalent_sample_size

        self._dag = None

    def fit(self, X_train, y_train, **fit_params):

        data = pd.concat([X_train, y_train], axis=1)
        graph_search_est = self.graph_search_algo(data)
        if self.graph_search_algo == HillClimbSearch:
            parameters = dict(
                scoring_method=self.scoring_method,
                start_dag=self.start_dag,
                fixed_edges=self.fixed_edges,
                tabu_length=self.tabu_length,
                max_indegree=self.max_indegree,
                black_list=self.black_list,
                white_list=self.white_list,
                epsilon=self.epsilon,
                max_iter=self.max_iter,
                show_progress=self.show_progress
            )
        elif self.graph_search_algo == PC:
            parameters = dict(
                variant=self.variant,
                ci_test=self.ci_test,
                max_cond_vars=self.max_cond_vars,
                return_type=self.return_type,
                significance_level=self.significance_level,
                show_progress=self.show_progress
            )
        dag = graph_search_est.estimate(**parameters)

        

        extra_columns = list(set(data.columns) - set(dag.nodes))
        if y_train.name in extra_columns:
            raise ValueError('Resulting DAG does not contain target. It cannot be used to make predictions.')
        if len(extra_columns) > 0:
            data = data.drop(columns=extra_columns)
        elif len(extra_columns) < 0:
            raise ValueError('Invalid value for extra_columns')
        
        self._dag = dag
        self.ebunch = list(dag.nodes)
        super().__init__(dag)
        print('Now fitting the graph...')
        super().fit(
            data, 
            estimator=self.estimator, 
            prior_type=self.prior_type, 
            pseudo_counts=self.pseudo_counts,
            equivalent_sample_size=self.equivalent_sample_size,
            **fit_params
        )
        print('Succesfully fitted the graph')

        self.X_ = X_train
        self.y_ = y_train
        self.fit_params_ = fit_params
        self.classes_ = LabelEncoder().fit(y_train).classes_

        return self

    def predict(self, X, stochastic=False, n_jobs=-1):
        extra_columns = list(set(X.columns) - set(self.nodes))
        if len(extra_columns) > 0:
            X = X.drop(columns=extra_columns)
        elif len(extra_columns) < 0:
            raise ValueError('Invalid value for extra_columns')
        y_pred_df = super().predict(X, stochastic, n_jobs)
        y_pred = y_pred_df.to_numpy()
        self.y_pred_ = y_pred

        return y_pred

    def predict_proba(self, X):
        extra_columns = list(set(X.columns) - set(self.nodes))
        if len(extra_columns) > 0:
            X = X.drop(columns=extra_columns)
        elif len(extra_columns) < 0:
            raise ValueError('Invalid value for extra_columns')
        y_pred_proba_df = super().predict_probability(X)
        y_pred_proba = y_pred_proba_df.to_numpy()
        self.y_pred_proba_ = y_pred_proba
        
        return y_pred_proba
    
    def get_params(self, deep=True):
        return {
            'ebunch':self.ebunch,
            'graph_search_algo':self.graph_search_algo,
            'scoring_method':self.scoring_method,
            'start_dag':self.start_dag,
            'fixed_edges':self.fixed_edges,
            'tabu_length':self.tabu_length,
            'max_indegree':self.max_indegree,
            'black_list':self.black_list,
            'white_list':self.white_list,
            'epsilon':self.epsilon,
            'max_iter':self.max_iter,
            'show_progress':self.show_progress,
            'estimator':self.estimator,
            'prior_type':self.prior_type,
            'pseudo_counts':self.pseudo_counts,
            'equivalent_sample_size':self.equivalent_sample_size
        }
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

In [ ]:
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate

def cv_scorer(ml_algo, scoring, *data, model_name=None, algo_params={}, resampler=None, output=True):    
    if len(data) == 2:
        X, y = tuple(data)
        w = None
    elif len(data) == 3:
        X, y, w = tuple(data)
    else:
        print('Invalid length for "data".')
        return
    
    if resampler != None:
        model = Pipeline([('Resampling', resampler()), (model_name, ml_algo(**algo_params))])
    else:
        model = Pipeline([(model_name, ml_algo(**algo_params))])
    
    cv_scores = cross_validate(model, X, y, scoring=scoring, fit_params={}, return_estimator=True)
    if output == True:
        if type(model_name) != type(None):
            print(model_name + ' cv_scores:')
        else:
            print('cv_scores:')
        print(cv_scores)
        print()
    
    cv_scores_summary = {}
    cv_scores_summary['estimator'] = cv_scores['estimator']
    for score in scoring:
        scores_ = cv_scores['test_' + score]
        mean_ = cv_scores['test_' + score].mean()
        std_ = cv_scores['test_' + score].std()
        
        cv_scores_summary[score]= dict(zip(['scores', 'mean', 'std'], [scores_, mean_, std_]))
        
        if output == True:
            print(score + ' mean: ' + f'{mean_:0.2f}')
            print(score + ' std: ' + f'{std_:0.4f}')
            print()
    if print == True:
        print()
    
    if type(model_name) != type(None):
        return {model_name:cv_scores_summary}, cv_scores
    else:
        return cv_scores_summary, cv_scores

In [ ]:
included_columns = ['Good_Health', 'Hypertension', 'High_Cholesterol', 'Smoker_Status', 'Age_Cat', 'Diabetes', 'Sodium', 'Heavy_Drinker', 'Heart_Disease', 'SEX', 'Sample_Weights']
df = df[included_columns]
df

In [ ]:
df['Heart_Disease'] = df['Heart_Disease'].cat.remove_unused_categories()
df = df.astype(int).astype(str)

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Heart_Disease', 'Sample_Weights'])
y = df['Heart_Disease']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=42)

X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
def no_parents_black_list(list_orig, list_no_parents):
    '''Given a list of nodes, list_orig, and another list of nodes that should not have parents, 
    list_no_parents, generates the required black_list for use in graph search algorithms'''

    list_start = []
    list_end = []
    for node_no_parent in list_no_parents:
        list_temp_1 = list_orig.copy()
        list_temp_1.remove(node_no_parent)
        list_temp_2 = [node_no_parent] * len(list_temp_1)
        list_start.append(list_temp_1)
        list_end.append(list_temp_2)
    assert len(list_start) == len(list_end)
    
    black_list = []
    for i in range(len(list_start)):
        black_list = black_list + list(zip(list_start[i], list_end[i]))

    return black_list



In [ ]:
from sklearn.metrics import roc_curve

def find_threshold(fpr_chosen):
    fpr, _, thresholds= roc_curve(y_test, y_proba.iloc[:, 1])
    indices = np.where(fpr <= fpr_chosen)[0]
    index = indices[-1:-2:-1][0] + 1
    return thresholds[index]

def predict_with_treshold(proba, threshold):
    return (proba >= threshold).astype(float)

In [ ]:
black_list = no_parents_black_list(list(pd.concat([X, y], axis=1).columns), ['SEX', 'Age_Cat'])
black_list

In [ ]:
path = './cpdag.pkl'
if os.path.exists(path):
    with open(path, 'rb') as f:
        cpdag = pickle.load(f)
        print('Loaded cpdag')
else:
    pc_algo = PC(pd.concat([X_train, y_train], axis=1))
    cpdag = pc_algo.estimate(return_type='cpdag')
    print('cpdag built')

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

nx.draw_shell(cpdag, with_labels=True)
plt.show()

In [ ]:
path = './bm_predictions.pkl'
if os.path.exists(path):
    with open(path, 'rb') as f:
        _ = pickle.load(f)
        bm = pickle.load(f)
        y_test = pickle.load(f)
        y_pred = pickle.load(f)
        y_proba = pickle.load(f)
        print(f'Loaded from {path}.')
else:
    from pgmpy.estimators import BayesianEstimator

    bm = BayesianModel(graph_search_algo=HillClimbSearch, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=5, black_list=black_list)
    bm.fit(X_train, y_train)
    y_pred = bm.predict(X_test)
    y_proba = bm.predict_proba(X_test)
    with open(path, 'wb') as f:
        pickle.dump(4, f)
        pickle.dump(bm, f)
        pickle.dump(y_test, f)
        pickle.dump(y_pred, f)
        pickle.dump(y_proba, f)
    print('Model trained.')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay

RocCurveDisplay.from_predictions(y_test, y_proba[:, 1], pos_label='1')
plt.show()
PrecisionRecallDisplay.from_predictions(y_test, y_proba[:, 1], pos_label='1')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

print(classification_report(y_test, y_pred))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, normalize='true')
plt.show()

In [ ]:
import networkx as nx

nx.draw_shell(bm, with_labels=True)

In [ ]:
from pgmpy.estimators import BayesianEstimator
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import StratifiedKFold

roc_auc_scorer = make_scorer(roc_auc_score, needs_proba=True)

scores = cross_validate(BayesianModel(graph_search_algo=HillClimbSearch, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=5), 
    X, y, scoring=['recall', 'roc_auc'], cv=StratifiedKFold())
scores

In [ ]:
cv_scores_summary, cv_scores = cv_scorer(BayesianModel, ['recall'], X_train, y_train, model_name='BayesianModel', algo_params=dict(graph_search_algo=HillClimbSearch, \
    graph_search_params={}, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=5), resampler=None, output=True)

In [ ]:
from pgmpy.inference.CausalInference import CausalInference

inference = CausalInference(bm)

In [ ]:
dict_parents = {}
for parent in bm.get_parents('Heart_Disease'):
    dict_parents[parent]= list(X_train[parent].unique())
dict_parents

In [ ]:
phi_dict = {}
phi_dict['No intervention'] = inference.query(['Heart_Disease'])
for parent, values in dict_parents.items():
    for value in values:
        name = f'{parent}_{value}'
        phi_dict[name] = inference.query(['Heart_Disease'], do={parent:value})
phi_dict['Hypertension_2.0, High_Cholesterol_2.0'] = inference.query(['Heart_Disease'], do={'Hypertension':'2', 'High_Cholesterol':'2'})

In [ ]:
def discrete_factor_to_series(factor, name=None):
    variables = factor.scope()
    n_states = 1
    for cardinality in factor.get_cardinality(variables).values():
        n_states = n_states * cardinality
    state_indices = list(range(n_states))
    assignments = factor.assignment(state_indices)
    state_names = []
    for assignment in assignments:
        state_name = ''
        for tuple_ in assignment:
            state_name = state_name +  '__' + tuple_[0] + '_' + tuple_[1]
        state_names.append(state_name)
    states_dicts = [{state[0]:state[1] for state in assignment} for assignment in assignments]
    values = [round(factor.get_value(**dict_), 4) for dict_ in states_dicts]

    return pd.Series(values, index=state_names, name=name)

In [ ]:
series_list = []
for name, factor in phi_dict.items():
    series_list.append(discrete_factor_to_series(factor, name))

infer_df = pd.DataFrame()
columns = []
for series in series_list:
    columns.append(series.name)
    infer_df = pd.concat([infer_df, series], axis=1)
infer_df.columns = columns
infer_df = infer_df.T

In [ ]:
infer_df

**Testing Active Trail Function**

In [ ]:
toy_model = BayesianNetwork([('Difficulty', 'Grade'), ('Intelligence', 'Grade'),('Intelligence', 'SAT'), ('Grade', 'Letter')])

In [ ]:
toy_model.get_random_cpds(n_states = {'Difficulty':2, 'Intelligence':2, 'Grade':5, 'SAT':2, 'Letter':2}, inplace=True)

In [ ]:
help(BayesianNetwork)

In [ ]:
nx.draw_kamada_kawai(toy_model, with_labels=True)

In [ ]:
for node in toy_model.nodes:
    print(toy_model.active_trail_nodes(node, []))

In [ ]:
toy_model.nodes

In [ ]:
toy_model.states

In [ ]:
df_toy = pd.DataFrame(index=[0])

In [ ]:
df_toy

In [ ]:
toy_model.predict_probability(df_toy)

In [ ]:
toy_model.active_trail_nodes(['SAT'])